In [ ]:
import dxpy
import dxdata
import pandas as pd
import pyspark
import subprocess
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext

dxdata.__version__
connection = dxdata.connect()
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "2000m")

In [ ]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [ ]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
dataset.entities

In [ ]:
participant = dataset['participant']

In [ ]:
# Returns all field objects for a given UKB showcase field id
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id
def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

In [ ]:
# split the recommended fields into a list
merged_fields = pd.read_csv("merged_fields.tsv", sep="\t", header=0)
merge_field_list = merged_fields["field_id"].tolist()
# cut the recommended fields list for 20 fields each group
merge_field_list = [merge_field_list[i:i + 20] for i in range(0, len(merge_field_list), 20)]
for i, group in enumerate(merge_field_list):
    if i <= 13:
        continue
    print(f"Retrieving fields for group {i + 1} with {len(group)} fields")
    # change the group astype to string
    group = [str(x) for x in group]
    get_ids = sum([field_names_for_id(field_id) for field_id in group], [])
    field_names = ['eid'] + get_ids
    df = participant.retrieve_fields(names=field_names, engine=connection)
    data = df.toPandas()
    display(data)
    # save the data to a file
    # if the file already exists, continue to the next iteration
    data.to_csv(f'./core_category/fields_group_{i + 1}.tsv', sep='\t', index=False, header=True)
    # Upload the file using subprocess
    subprocess.run([
        "dx", "upload", f"./core_category/fields_group_{i + 1}.tsv",
        "-p", "--path", "/Output/Traits/core_category/", "--brief"
    ], check=True)